This notebook is designed to get users up-to-speed with the capabilities of molsim's Simultaneous Autonomous Molecular Emission Resolver (SAMER). 
This routine is intended to measure the excitation temperature, linewidth, velocity, and column densities for a given list of molecules to a given spectrum.
This routine was tested on spectra from a number of star-forming regions which are typically a dense collection of emission lines.
The intended use case is to fit the spectra extacted from an entire field of view on a per-pixel basis, allowing the user to create maps of physical quantites.
The spectra are required to be in a specific format, please see the included extract_spectra.py for a template script.

In [ ]:
#Quick way to import necessary molsim functions
%run -i /Users/samer/Documents/nrao_dir/molsim_dev/ipython_quickstart.py

from molsim.samer import *

# from molsim.samer.SAMER_Classes import Fitting_Variables, Pixel, Molecule_Parameters
# from molsim.samer.SAMER_Functions import file_to_dict, gen_continuum, gen_pixel_dict, pixel_hopping, create_params, store_params, read_params, update_params
# from molsim.samer.SAMER_Functions import calc_mean_params, obs_exclusion, min_function, fit_initial_column, fit_row, mp_helper, fit_multiprocessing
# from molsim.samer.SAMER_Functions import plot_fit, plot_fit_multi, print_lines_from_fit, pull_fits_header_info, create_figure

import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
from molsim.utils import _get_res
import json
import time
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import glob
import matplotlib
import pathlib
import lmfit
from lmfit import Model, Parameters, Minimizer, report_fit
import os
import multiprocessing as mp
%matplotlib widget

In [ ]:
#Load in all of the molecule data into the appropriate Molecule object for molsim
census_directory = '/Users/samer/Downloads/2018_census_catalogs/'

nh2cn = load_mol(census_directory+'nh2cn.cat', type='SPCAT', name='nh2cn', qpart_file=census_directory+'nh2cn.qpart')
_13ch3oh = load_mol(census_directory+'13ch3oh_brett.cat', type='SPCAT', name='_13ch3oh', qpart_file=census_directory+'13ch3oh.qpart')
ch3cn = load_mol(census_directory+'ch3cn.cat', type='SPCAT', name='ch3cn', qpart_file=census_directory+'ch3cn.qpart')
ch3cn_1v8 = load_mol(census_directory+'ch3cn_1v8.cat', type='SPCAT', name='ch3cn_1v8', qpart_file=census_directory+'ch3cn.qpart')
nh2cho = load_mol(census_directory+'nh2cho.cat', type='SPCAT', name='nh2cho', qpart_file=census_directory+'nh2cho.qpart')
nh2cho_1v12 = load_mol(census_directory+'nh2cho_1v12.cat', type='SPCAT', name='nh2cho_1v12', qpart_file=census_directory+'nh2cho_1v12.qpart')
ch3cho = load_mol(census_directory+'ch3cho.cat', type='SPCAT', name='ch3cho', qpart_file=census_directory+'ch3cho.qpart')
ch3ocho = load_mol(census_directory+'ch3ocho.cat', type='SPCAT', name='ch3ocho', qpart_file=census_directory+'ch3ocho.qpart')
ch3cooh = load_mol(census_directory+'ch3cooh.cat', type='SPCAT', name='ch3cooh', qpart_file=census_directory+'ch3cooh.qpart')
ch3cooh_v1 = load_mol(census_directory+'ch3cooh_v1.cat', type='SPCAT', name='ch3cooh_v1', qpart_file=census_directory+'ch3cooh.qpart')
hcoch2oh = load_mol(census_directory+'hcoch2oh.cat', type='SPCAT', name='hcoch2oh', qpart_file=census_directory+'hcoch2oh.qpart')
ch3och3 = load_mol(census_directory+'ch3och3.cat', type='SPCAT', name='ch3och3', qpart_file=census_directory+'ch3och3.qpart')
c2h5oh = load_mol(census_directory+'c2h5oh.cat', type='SPCAT', name='c2h5oh', qpart_file=census_directory+'c2h5oh.qpart')
c2h5cn = load_mol(census_directory+'c2h5cn.cat', type='SPCAT', name='c2h5cn', qpart_file=census_directory+'c2h5cn.qpart')
ch3coch3 = load_mol(census_directory+'ch3coch3.cat', type='SPCAT', name='ch3coch3', qpart_file=census_directory+'ch3coch3.qpart')
a_ch2ohch2oh = load_mol(census_directory+'a-ch2ohch2oh.cat', type='SPCAT', name='a_ch2ohch2oh', qpart_file=census_directory+'a-ch2ohch2oh.qpart')
g_ch2ohch2oh = load_mol(census_directory+'g-ch2ohch2oh.cat', type='SPCAT', name='g_ch2ohch2oh', qpart_file=census_directory+'g-ch2ohch2oh.qpart')
so2 = load_mol(census_directory+'so2.cat', type='SPCAT', name='so2', qpart_file=census_directory+'so2.qpart')
so2_v2 = load_mol(census_directory+'so2_v2.cat', type='SPCAT', name='so2_v2', qpart_file=census_directory+'so2.qpart')
ch3oh = load_mol(census_directory+'ch3oh.cat', type='SPCAT', name='ch3oh', qpart_file=census_directory+'ch3oh.qpart')
so = load_mol(census_directory+'so.cat', type='SPCAT', name='so', qpart_file=census_directory+'so.qpart')
so_v1 = load_mol(census_directory+'so_v1.cat', type='SPCAT', name='so_v1', qpart_file=census_directory+'so.qpart')
h13cn = load_mol(census_directory+'h13cn.cat', type='SPCAT', name='h13cn', qpart_file=census_directory+'h13cn.qpart')
h13co_p = load_mol(census_directory+'h13co+.cat', type='SPCAT', name='h13co_p', qpart_file=census_directory+'h13co+.qpart')
hc15n = load_mol(census_directory+'hc15n.cat', type='SPCAT', name='hc15n', qpart_file=census_directory+'hc15n.qpart')
ch2dcn = load_mol(census_directory+'ch2dcn.cat', type='SPCAT', name='ch2dcn', qpart_file=census_directory+'ch2dcn.qpart')
oc34s = load_mol(census_directory+'oc34s.cat', type='SPCAT', name='oc34s', qpart_file=census_directory+'oc34s.qpart')

In [ ]:
#Put the Molecule objects that we want to fit to the data in a list
#Note that if there are molecules we want to fit as a set, we only include one of those species here
fit_molecules = [a_ch2ohch2oh, g_ch2ohch2oh, c2h5oh, c2h5cn, ch3coch3,
                ch3och3, ch3ocho, ch3cooh, hcoch2oh,
                ch3oh, _13ch3oh, ch3cho, ch3cn, ch2dcn, 
                nh2cho, nh2cn, h13cn, hc15n, h13co_p, 
                so, so2, oc34s]

#Need to organize the list of molecules into a properly-formatted dictionary for the fit
#Molecules that you want to fit with the same parameters should be associated with the same key
#In this case we are fitting the ground state with the vibrational state for a number of molecules
fit_molecules_dict = dict()
for mol in fit_molecules:
    if mol.name == 'ch3cooh':
        fit_molecules_dict[mol.name] = [ch3cooh, ch3cooh_v1]
    elif mol.name == 'ch3cn':
        fit_molecules_dict[mol.name] = [ch3cn, ch3cn_1v8]
    elif mol.name == 'nh2cho':
        fit_molecules_dict[mol.name] = [nh2cho, nh2cho_1v12] 
    elif mol.name == 'so':
        fit_molecules_dict[mol.name] = [so, so_v1]
    elif mol.name == 'so2':
        fit_molecules_dict[mol.name] = [so2, so2_v2]
    else:
        fit_molecules_dict[mol.name] = [mol]
        
#Set the upper state energy threshold for excluding certain molecular transitions from the fit
#Use 'exclude' to mask all the channels in your observations with emission from the molecule
#Else pass a number (n) to exclude all transitions with Eup < n * mean background temperature for a pixel
exclusion_dict = dict()
for mol in fit_molecules_dict:
    if mol == 'ch3oh':
        exclusion_dict[mol] = 'exclude'
    elif mol == 'h13cn':
        exclusion_dict[mol] = 'exclude'
    elif mol == 'hc15n':
        exclusion_dict[mol] = 'exclude'
    elif mol == 'h13co_p':
        exclusion_dict[mol] = 'exclude'
    elif mol == 'so':
        exclusion_dict[mol] = 'exclude'
    elif mol == 'oc34s':
        exclusion_dict[mol] = 'exclude'
    elif mol == 'example_mol':
        exclusion_dict[mol] = 3.

In [ ]:
#Defining the attributes of our Fitting_Variables object so that we can begin the fit
#Attribute descriptions are available in SAMER_classes.py
example_fv = Fitting_Variables()

example_fv.source = 'g34.41'
example_fv.spectra_dir = '/Users/samer/Downloads/coccoa_spectra/'+example_fv.source+'_complete/'
example_fv.output_dir = '/Users/samer/Downloads/coccoa_results/'+example_fv.source+'_results/'
example_fv.x_size = 45
example_fv.y_size = 40
example_fv.observatory = Observatory(sd=False, array=True, synth_beam=[0.33,0.33])
example_fv.shared_tex = True
example_fv.shared_dV = True
example_fv.shared_vlsr = True
example_fv.tex_calculation = 'minimizer'
example_fv.dV_calculation = 'minimizer'
example_fv.molecule_dict = fit_molecules_dict
example_fv.initial_tex = 110 
example_fv.initial_dV = 4.4
example_fv.initial_vlsr = 58
example_fv.initial_nt = 17.5
example_fv.tex_flex = 10 #20
example_fv.dV_flex = 0.3
example_fv.vlsr_flex = 0.7
example_fv.nt_flex = 0.4
example_fv.nt_flex_pixel_1 = 2.5
example_fv.exclusion = True
example_fv.exclusion_dict = exclusion_dict

#Pass customized initial parameters for molecules
custom_params_dict = dict()
for mol in fit_molecules_dict:
    if mol == 'ch3oh':
        custom_params_dict[mol] = dict()
        custom_params_dict[mol]['nt'] = 18.
example_fv.custom_params_dict = custom_params_dict

# example_fv.min_sep = 5 #The minimum separation between peaks (in MHz) during the peak finding process
# example_fv.sigma = 3 #The sigma cutoff for detecting peaks by the peakfinder (i.e. 3 for 3 sigma, etc.)
# example_fv.n_chans = 2 #The number of channels to include on either side of the central channel when fitting Gaussians to the peaks
# example_fv.n_bins = 100 #The number of bins to include in the histogram of FWHMs

start_pixel = '20_13'

In [ ]:
#Generate a dictionary of Pixel objects so that we can more easily pick out spectra to view in the notebook
pixel_dict = gen_pixel_dict(example_fv)

In [ ]:
#Begin the fitting
fit_initial_column(pixel_dict, example_fv, start_pixel=start_pixel, single_pixel=False)

In [ ]:
#If one wants to fit a single row
#fit_row(7, pixel_dict, example_fv, start_pixel=start_pixel)

In [ ]:
#If one wants to fit the entire image while utilizing multiple cores of the CPU
rows = np.arange(0, example_fv.y_size)
if __name__ == '__main__':
    fit_multiprocessing(rows, mp.cpu_count()-1, start_pixel, pixel_dict, example_fv)

In [ ]:
#Alternatively pull the information for Pixel objects from output files if they were previously fit
for pixel in pixel_dict:
    if os.path.isfile(example_fv.output_dir+example_fv.source+'_'+pixel+'.txt'):
        read_params(pixel, pixel_dict, example_fv)

In [ ]:
#Plot the resulting fit for a single pixel
if 'fig' in locals():
    plt.close(fig)

plot_pixel = '30_9'
plot_mols = ['ch3ocho', 'hcoch2oh'] #If one wants to highlight a specific molecule or set of molecules
plot_fit(plot_pixel, pixel_dict, example_fv, show_sum=True, highlight_molecules=plot_mols, figsize=(10,5))
for mol in plot_mols:
    #Print out the fit parameters for the selected list of molecules
    print(mol, pixel_dict[plot_pixel].molecule_params[mol].tex, pixel_dict[plot_pixel].molecule_params[mol].dV, 
          pixel_dict[plot_pixel].molecule_params[mol].vlsr, pixel_dict[plot_pixel].molecule_params[mol].nt)
    #Output a table with the detected transitions in your frequency range for the selected list of molecules
    print_lines_from_fit(plot_pixel, pixel_dict, example_fv, mol, tau=None, eup=None, int_thresh=0.001)

In [ ]:
#Plot multiple pixels at once if one wants to compare spectra/fits
if 'fig' in locals():
    plt.close(fig)

plot_pixel_list = ['17_8', '19_30']

plot_mols = ['ch3ocho']

plot_fit_multi(plot_pixel_list, pixel_dict, example_fv, show_sum=True, highlight_molecules=plot_mols)

for plot_pixel in plot_pixel_list:
    mol = plot_mols[0]
    print(plot_pixel, pixel_dict[plot_pixel].molecule_params[mol].tex, pixel_dict[plot_pixel].molecule_params[mol].dV, 
          pixel_dict[plot_pixel].molecule_params[mol].vlsr, pixel_dict[plot_pixel].molecule_params[mol].nt)
    #print_lines_from_fit(plot_pixel, pixel_dict, example_fv, mol, tau=None, eup=None, int_thresh=0.001)

In [ ]:
#Create figures and .fits files with the correct coordinates
#Requires a template .fits file that contains the necessary information in its header
fits_dir = '/Users/samer/Downloads/coccoa_template_fits_files/'
infile = fits_dir + 'G34.41+0.24_TuneB.cont_multiR0.5_ap.image.tt0.pbcor.0.33.fits'
pos_key = example_fv.spectra_dir+example_fv.source+'_pos_key.txt'
fits_info = pull_fits_header_info(infile, pos_key, example_fv)

In [ ]:
#Create the continuum file first so contours can be applied to other figures
create_figure('continuum', pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5))

In [ ]:
#Create a set of figures with no masking criteria
tbg_contours = [15,30,45]
create_figure('tex', pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5), contours=tbg_contours)
create_figure('dV', pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5), contours=tbg_contours)
create_figure('vlsr', pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5), contours=tbg_contours)
for mol in pixel_dict['0_0'].molecule_params:
    create_figure(mol, pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5), contours=tbg_contours)

In [ ]:
#Create a set of figures with a variety of masking criteria
tbg_contours = [15,30,45]
create_figure('tex', pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5), contours=tbg_contours,
             tex_thresh_upper=130.)
create_figure('dV', pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5), contours=tbg_contours,
             tex_thresh_upper=130.)
create_figure('vlsr', pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5), contours=tbg_contours,
             tex_thresh_upper=130.)
for mol in pixel_dict['0_0'].molecule_params:
    create_figure(mol, pixel_dict, example_fv, make_fits=True, fits_info=fits_info, make_png=True, figsize=(6,5), contours=tbg_contours,
                 tex_thresh_upper=130., transition_thresh=2, noise_thresh=3*get_rms(pixel_dict['0_0'].obs_y))

In [ ]:
#Create a set of ratio maps with respect to a selected molecule
ratio_mol = 'ch3ocho'
for mol in pixel_dict['0_0'].molecule_params:
    create_figure('ratio', pixel_dict, example_fv, make_fits=True, fits_info=fits_info, ratio_mols=[mol,ratio_mol], make_png=True, figsize=(6,5), contours=[12],
                 tex_thresh_upper=130.)